In [ ]:
import numpy as np
import scipy
import scipy.spatial
from scipy.spatial.distance import squareform,pdist,cdist
import matplotlib.pyplot as plt
from scipy.io import mmread

from sklearn.manifold import TSNE
from joblib import Parallel, delayed

from timeit import default_timer as timer

import scanpy as sc
import anndata as ad


from scipy.sparse import csr_matrix
import pandas as pd
import anndata as ad


In [ ]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('..'))


In [ ]:
# Load data and extracted angle information from Experiment2_Dermal_5K_Genes.ipynb
# (The experiment that partitioned the three figurine images into 3 partitions (K=3))
f= np.load('',allow_pickle=True).item()


In [ ]:
x= f['x']
omega= f['omega']
feature_partitions= f['feature_partitions']
c_yoda= f['color_yoda']
c_dog= f['color_dog']
c_bunny= f['color_bunny']
yoda_circ =f['yoda_circ']
dog_circ =f['dog_circ']
bunny_circ =f['bunny_circ']
del f

In [ ]:
K=3

### Multi-tSNE

In [ ]:
# Clone the py_mmtsne repository (uncomment to execute)
# !git clone https://github.com/gsaygili/py_mmtsne.git

# This code uses the repository at commit: fd8d39ed43b254cbb9bf0e776a9a394e894a4b40


In [ ]:
from sklearn.manifold import _utils
from sklearn.metrics.pairwise import _VALID_METRICS, pairwise_distances

# The function extracts the joint probability matrix P from t-SNE based on the given data.
#
# Input:
#   data : ndarray of shape (N, D)
#       Input data matrix with N samples and D features.
#
#   perplexity : float
#       Perplexity parameter used to set the bandwidth of the conditional distributions in t-SNE.
#
# Output:
#   P : ndarray of shape (N, N)
#       Symmetrized joint probability matrix used in the t-SNE objective.

def get_tSNE_P(data,perplexity):
    distances = pairwise_distances(data, squared=True).astype(np.float32, copy=False)
    conditional_P = _utils._binary_search_perplexity(distances, perplexity, False)
    P = conditional_P + conditional_P.T
    P = np.maximum(P / np.sum(P), np.finfo(float).eps)
    return P


In [ ]:
from py_mmtsne.multmaps_tsne import mult_maps_tsne 


In [ ]:
# Generate two embeddings for each parameter setting, where each parameter is a tuple: 
# the first element specifies the perplexity, and the second defines the embedding dimension used in the generated embeddings

multi_tsne_params= np.array([[5,2],[10,2],[20,2],[40,2],[5,3],[10,3],[20,3],[40,3]])

multi_tsne = [mult_maps_tsne( get_tSNE_P(x,multi_tsne_params[i][0]), no_maps=K, no_dims=multi_tsne_params[i][1])[0]  for i in range(len(multi_tsne_params))]

### IC- PML

In [ ]:
# Clone the ic-pml repository (uncomment to execute)
# !git clone https://github.com/he-jesse/ic-pml.git

# This code uses the repository at commit: 527c7d9169db88913cd1dc5e417355a1d68ae18f

In [ ]:
# The code was modified to ensure it runs without errors:
# - Commented out `load_latex_preamble()` in `ic-pml/_configure.py`
# - Commented out `import matlab.engine` in `methods.py`

In [ ]:
# Importing functions from `methods.py` in the `ic-pml` subdirectory
# by temporarily changing the working directory
#
# import os
# os.chdir('ic-pml')
# from methods import *
# os.chdir('..')

In [ ]:
from scipy.spatial.distance import cdist

# Computes the largest Euclidean distance from any point to one of its k-nearest neighbors
def get_max_knn(dist_mat, k):
    return np.max(np.partition(dist_mat,1+k)[:,1+k])
    
# The maximal embedding dimnsion in each partition
ic_pml_max_emb_dim=5

# Compute the Euclidean distance matrix
cur_dis_matrix= cdist(x,x)**2

eps_arr= [get_max_knn(cur_dis_matrix,i) for i in [5,10,20,40]]
sim_crit= [0.01,0.03,0.05, 0.1,0.3,0.5]
eig_crit= [0.01,0.03,0.05, 0.1,0.3,0.5,1,2]


# Generate three embeddings for each parameter setting.
# Each parameter is a triplet where:
# * the first element specifies the bandwidth of the affinity matrix,
# * the second defines the similarity criterion value,
# * and the third sets the eigenvalue criterion.

embs_ic_pml= np.zeros(( len(eps_arr)*len(sim_crit)*len(eig_crit),K,len(x),ic_pml_max_emb_dim))
embs_ic_pml_dims= np.zeros(( len(eps_arr)*len(sim_crit)*len(eig_crit),K))

for i,(eps_used, sim_crit_used,eig_crit_used) in enumerate([  (a,b,c) for a in eps_arr for b in sim_crit for c in eig_crit]):
    print(str(i)+' '+str(len(eps_arr)* len(sim_crit)*len(eig_crit)))
    
    fact = factorize(x, sigma=eps_used, n_factors = K, sim_crit=sim_crit_used, eig_crit=eig_crit_used, n_eigenvectors=50)

    for s in range(K):
        eigs_used = np.array(fact['manifolds'][s][:ic_pml_max_emb_dim])
        embs_ic_pml[i,s,:,:len(eigs_used)]=  fact['phi'][:, eigs_used]
        embs_ic_pml_dims[i,s]= len(eigs_used)
            


In [ ]:
ic_pml_params= np.array([  (a,b,c) for a in eps_arr for b in sim_crit for c in eig_crit])

### LDLE

In [ ]:
from pyLDLE2 import buml_, util_
from scipy.sparse.csgraph import floyd_warshall, shortest_path
from sklearn.neighbors import NearestNeighbors

# The function computes the k-nearest neighbors for each sample, ordered from closest to farthest,
# based on a given pairwise distance matrix.
#
# Input:
#   dist_mat : ndarray of shape (N, N)
#       Pairwise distance matrix where dist_mat[i, j] is the distance between sample i and sample j.
#
#   k : int
#       Number of nearest neighbors to retrieve for each sample.
#
# Output:
#   indsA : ndarray of shape (N, k)
#       Each row contains the indices of the k nearest neighbors for that sample,
#       sorted from the closest to the farthest neighbor.
def get_K_min_inds_sorted(dist_mat,k=50):
    indsA = np.argpartition(dist_mat+np.eye(len(dist_mat))*np.max(dist_mat),k,axis=1)[:,:k]
        
    indsB = np.argsort(dist_mat[np.arange(len(dist_mat))[:,None], indsA],axis=1)
    indsA= indsA[np.arange(len(A))[:,None], indsB]
    return indsA

# Copied from `pyLDLE2/util_.py` to identify nearest neighbors in a torned embedding,
# using the same approach as in the original paper’s analysis.
def shortest_paths(X, n_nbrs):
    nbrs = NearestNeighbors(n_neighbors=n_nbrs).fit(X)
    knn_graph = nbrs.kneighbors_graph(mode='distance')
    dist_matrix, predecessors = shortest_path(knn_graph, return_predecessors=True, directed=False)
    return dist_matrix, predecessors


# Defined in order to allow the code to run
np.int= int


# Generate a single embedding for each parameter setting.
# Each parameter is a quadruplet where:
#   * the first element specifies the eta parameter,
#   * the second defines the k_tune parameter,
#   * the third sets the embedding dimension,
#   * and the fourth indicates whether the embedding should be torned.
#
# Then, extract the k-nearest neighbors of each point based on the generated embedding.
# For the torned case, we follow the same procedure as in `compute_global_distortions()`
# from `pyLDLE2/util_.py`.

LDLE_parameters= np.zeros((32,4))
LDLE_embs= []
LDLE_nn=[]
s= 0 
for eta in [5,10]:
    for k_tune in [5,10,20,40]:
        for d in [2,3]:
            for is_torn in [False, True]:
                print('------------***********************')
                print(str(eta)+'   '+str(d)+'   '+str(is_torn))
                LDLE_parameters[s,:]= np.array([eta,d,is_torn,k_tune])
                s+=1
                
                buml_obj = buml_.BUML(d=d, local_opts={'algo':'LDLE', 'k_tune':k_tune}, global_opts={'to_tear':is_torn},\
                          intermed_opts={'eta_min': eta})
                
                emb_ldle = buml_obj.fit(X= x)
                LDLE_embs.append(emb_ldle+0.)
                if is_torn:
                    n_nbr =10
                    y_s_d_e, _ = shortest_paths(buml_obj.GlobalViews.y_final+0., n_nbr)
                    intermed_param = buml_obj.IntermedViews.intermed_param
                    Utilde = buml_obj.IntermedViews.Utilde
                    C = buml_obj.IntermedViews.C
                    global_opts = buml_obj.global_opts
                    cur_dists = buml_obj.GlobalViews.compute_pwise_dist_in_embedding(intermed_param,
                                                                                Utilde, C, global_opts,
                                                                                dist=y_s_d_e, y=buml_obj.GlobalViews.y_final+0.)
                    
                else:
                    cur_dists= emb_ldle@emb_ldle.T
                    cur_dists= np.diag(cur_dists)[:,None]+np.diag(cur_dists)[None,:]-2*cur_dists

                
                LDLE_nn.append(get_K_min_inds_sorted(cur_dists))

# tSNE on all features

In [ ]:
# Generate a single embedding for each parameter setting.
# Each parameter is a pair where:
#   * the first element sets the embedding dimension,
#   * the second defines the perplexity parameter.

Tsne_all_params= np.array([[2,5],[2,10],[2,20],[2,40],[3,5],[3,10],[3,20],[3,40]])

Tsne_all = [TSNE(n_components=Tsne_all_params[i][0], perplexity=Tsne_all_params[i][1], random_state=42).fit_transform(x) for i in range(len(Tsne_all_params))]


# tSNE FP

In [ ]:
# Generate a single embedding for the data based on each of the extracted partitions

Tsne_FP= [TSNE(n_components=3, perplexity=10, random_state=42).fit_transform(x[:,feature_partitions[s]]) for s in range(K)]

# Evaluation of generated embeddings

### The following procedure is performed for each value of K (number of nearest neighbors).
###
### First, for each embedding, compute the K-nearest neighbor (KNN) error of each sample 
### with respect to each figurine’s angle configuration. The KNN error is defined as the number 
### of incorrectly estimated neighbors divided by K.
###
### Then, for each embedding and figurine, compute a score defined as the average KNN error 
### across all samples.
###
### Next, for each triplet of embeddings, compute the average score when each embedding is 
### paired with a distinct figurine. Take the minimal score across all valid matchings.
###
### Finally, for each value of K, return the triplet of embeddings with the lowest matching score.
###
### * In the case of single embeddings, the score is computed by matching the same embedding 
###   to each of the figurines individually and then averaging the resulting scores, 
###   as if the same embedding were used for all three figurines.


In [ ]:
from itertools import permutations
# Computes the percentage of intersection between two K-nearest neighbor lists for each point.
#
# Input:
#   neighbors : numpy.ndarray of shape (N, K)
#       First set of K-nearest neighbor indices for each of N points.
#
#   neighbors2 : numpy.ndarray of shape (N, K)
#       Second set of K-nearest neighbor indices for each of the same N points.
#
# Output:
#   result : numpy.ndarray of shape (N,)
#       For each point i, the number of overlapping neighbors in neighbors[i] and neighbors2[i],
#       divided by K (the number of neighbors).
def intersection_perc(neighbors,neighbors2):
    if neighbors.shape[0]!= neighbors2.shape[0] or neighbors.shape[1]!= neighbors2.shape[1]:
        raise Exception('The given matrices are not of the same size.')
        
    n= len(neighbors)
    res =np.zeros((n))
    for i in range(n):
        res[i]= len(set(neighbors[i]).intersection(set(neighbors2[i])))/len(neighbors[i])
    return res

# The function computes the k-nearest neighbors for each sample, ordered from closest to farthest,
# based on a given pairwise distance matrix.
#
# Input:
#   dist_mat : ndarray of shape (N, N)
#       Pairwise distance matrix where dist_mat[i, j] is the distance between sample i and sample j.
#
#   k : int
#       Number of nearest neighbors to retrieve for each sample.
#
# Output:
#   indsA : ndarray of shape (N, k)
#       Each row contains the indices of the k nearest neighbors for that sample,
#       sorted from the closest to the farthest neighbor.

def get_K_min_inds_sorted(dist_mat,k=50):
    
    indsA = np.argpartition(dist_mat+np.eye(len(dist_mat))*np.max(dist_mat),k,axis=1)[:,:k]
    
        
    indsB = np.argsort(A[np.arange(len(A))[:,None], indsA],axis=1)
    indsA= indsA[np.arange(len(A))[:,None], indsB]
    return indsA

amount_nn=np.arange(2,51,2)

true_nn = np.array( [get_K_min_inds_sorted(cdist(yoda_circ,yoda_circ), np.max(amount_nn)),\
                     get_K_min_inds_sorted(cdist(dog_circ,dog_circ), np.max(amount_nn)),\
                     get_K_min_inds_sorted(cdist(bunny_circ,bunny_circ), np.max(amount_nn))])



amount_labels=3

K=3
permute_types = np.array(list(permutations(range(K))))


# Multi-tSNE

In [ ]:
score_mult_tSNE_nn= np.zeros((len(amount_nn),len(multi_tsne),amount_labels,K,len(x)))

for r in range(len(multi_tsne)):
    for j in range(K):
        cur_dist= cdist( multi_tsne[r][:,:,j], multi_tsne[r][:,:,j])
        inds_mult_tSNE = get_K_min_inds_sorted(cur_dist, np.max(amount_nn))
        
        for i in range(amount_labels):
            for s in range(len(amount_nn)):
                score_mult_tSNE_nn[s,r,i,j,:]= intersection_perc(true_nn[i][:,:amount_nn[s]], inds_mult_tSNE[:,:amount_nn[s]])


In [ ]:
final_score_mult_tSNE= np.zeros((len(amount_nn), len(multi_tsne)))
for r in range(len(multi_tsne)):
       for i in range(len(amount_nn)):
            final_score_mult_tSNE[i,r]= np.max([ np.mean(score_mult_tSNE_nn[i,r,np.arange(K),permute_types[j] ]) for j in range(len(permute_types))])

In [ ]:
final_score_mult_tSNE_nn= np.max(final_score_mult_tSNE,axis=-1)
print(multi_tsne_params[np.argmax(final_score_mult_tSNE,axis=-1)])
print(final_score_mult_tSNE_nn)

# IC-PML

In [ ]:
score_icpml_nn = np.zeros((len(amount_nn),len(embs_ic_pml),amount_labels,K,ic_pml_max_emb_dim-1,len(x)))

for r in range(len(embs_ic_pml)):
    print(r)
    for j in range(K):

        for t,embs_ic_pml_dims in enumerate(np.arange(1,ic_pml_max_emb_dim)):
            if np.all(embs_ic_pml[r,j,:,embs_ic_pml_dims-1]==0.):
                continue
            
            cur_dist= cdist(embs_ic_pml[r,j,:,:embs_ic_pml_dims],embs_ic_pml[r,j,:,:embs_ic_pml_dims])
            inds_icpml = get_K_min_inds_sorted(cur_dist, np.max(amount_nn))
            
            for i in range(amount_labels):
                for s in range(len(amount_nn)):
                    score_icpml_nn[s,r,i,j,t,:]= intersection_perc(true_nn[i][:,:amount_nn[s]], inds_icpml[:,:amount_nn[s]])

best_score_icpml_nn_over_dims= np.max(np.mean(score_icpml_nn,axis=-1),axis=-1)
best_dims_per_icpml_sol = np.arange(1,ic_pml_max_emb_dim)[np.argmax(np.mean(score_icpml_nn,axis=-1),axis=-1)]

In [ ]:
final_score_icpml= np.zeros((len(amount_nn), len(embs_ic_pml)))
final_score_icpml_params= np.zeros((len(amount_nn), len(embs_ic_pml)))
for r in range(len(embs_ic_pml)):
       for i in range(len(amount_nn)):
            final_score_icpml[i,r]= np.max([ np.mean(best_score_icpml_nn_over_dims[i,r,np.arange(K),permute_types[j]])  for j in range(len(permute_types))])
            final_score_icpml_params[i,r]= np.argmax([ np.mean(best_score_icpml_nn_over_dims[i,r,np.arange(K),permute_types[j]])  for j in range(len(permute_types))])

In [ ]:
final_score_icpml_nn= np.max(final_score_icpml,axis=-1)
best_param_tot_ic_pml = ic_pml_params[np.argmax(final_score_icpml,axis=-1)]

print(ic_pml_params[np.argmax(final_score_icpml,axis=-1)])
print(final_score_icpml_nn)

# LDLE

In [ ]:
score_LDLE_nn= np.zeros((len(amount_nn),len(LDLE_nn),amount_labels,len(x)))

for s in range(len(amount_nn)):
    for j in range(len(LDLE_nn)):
        for i in range(amount_labels):        
            score_LDLE_nn[s,j,i,:]= intersection_perc(LDLE_nn[j][:,:amount_nn[s]], true_nn[i][:,:amount_nn[s]])
            
final_score_LDLE_nn= np.max( np.mean(np.mean(score_LDLE_nn,axis=-1),axis=-1),axis=-1)
print(final_score_LDLE_nn)

In [ ]:
print(LDLE_parameters[  np.argmax( np.mean(np.mean(score_LDLE_nn,axis=-1),axis=-1),axis=-1)])

# All features

In [ ]:
cur_dist = cdist(x,x)
score_all_nn= np.zeros((len(amount_nn), amount_labels, len(x)))

inds_all = get_K_min_inds_sorted(cur_dist, np.max(amount_nn))
for s in range(len(amount_nn)):
    for i in range(amount_labels):    
        score_all_nn[s,i,:]= intersection_perc(inds_all[:,:amount_nn[s]], true_nn[i][:,:amount_nn[s]])

final_score_all_nn= np.mean(np.mean(score_all_nn,axis=-1),axis=-1)
final_score_all_nn

# tSNE all

In [ ]:
score_tsne_all_nn= np.zeros((len(amount_nn),len(Tsne_all),amount_labels, len(x)))

for j in range(len(Tsne_all)):
    dist_all_tsne = cdist( Tsne_all[j],Tsne_all[j])
    inds_all_tsne = get_K_min_inds_sorted(dist_all_tsne, np.max(amount_nn))
    
    for i in range(amount_labels):
        for s in range(len(amount_nn)):
            score_tsne_all_nn[s,j,i,:]= intersection_perc(inds_all_tsne[:,:amount_nn[s]], true_nn[i][:,:amount_nn[s]])

final_score_tsne_all_nn= np.max(np.mean(np.mean(score_tsne_all_nn[:,:,:,:],axis=-1),axis=1),axis=-1)
print(Tsne_all_params[np.argmax(np.mean(np.mean(score_tsne_all_nn[:,:,:,:],axis=-1),axis=1),axis=-1)])
print(final_score_tsne_all_nn)

# FP Raw

In [ ]:
score_FP_raw_nn= np.zeros((len(amount_nn),amount_labels,K,len(x)))

for j in range(K):
    cur_x = x[:,feature_partitions[j]]
    ddd_FP_raw= cdist( cur_x,cur_x)
    inds_FP_raw = get_K_min_inds_sorted(ddd_FP_raw, np.max(amount_nn))
    
    for i in range(amount_labels):
        for s in range(len(amount_nn)):
            score_FP_raw_nn[s,i,j]= intersection_perc(inds_FP_raw[:,:amount_nn[s]], true_nn[i][:,:amount_nn[s]])


final_score_FP_raw_nn= np.zeros((len(amount_nn)))
for i in range(len(amount_nn)):
    final_score_FP_raw_nn[i]= np.max([ np.mean(score_FP_raw_nn[i,np.arange(K),permute_types[j] ]) for j in range(len(permute_types))])

In [ ]:
final_score_FP_raw_nn

# FP- tSNE

In [ ]:
score_FP_nn= np.zeros((len(amount_nn),amount_labels,K,len(x)))

for j in range(K):
    ddd_FP= cdist( Tsne_FP[j], Tsne_FP[j])
    inds_FP = get_K_min_inds_sorted(ddd_FP, np.max(amount_nn))
    
    for i in range(amount_labels):
        for s in range(len(amount_nn)):
            score_FP_nn[s,i,j]= intersection_perc(inds_FP[:,:amount_nn[s]], true_nn[i][:,:amount_nn[s]])


final_score_FP_nn= np.zeros((len(amount_nn)))
for i in range(len(amount_nn)):
    final_score_FP_nn[i]= np.max([ np.mean(score_FP_raw_nn[i,np.arange(K),permute_types[j] ]) for j in range(len(permute_types))])

In [ ]:
final_score_FP_nn

# Show results

In [ ]:
plt.rcParams["font.family"] = "DejaVu Serif"
plt.figure()
mark_size=80
plt.scatter(amount_nn, 1-final_score_tsne_all_nn,  marker='o',label='tSNE',alpha=0.8,c='red',s=mark_size*1.5)
plt.scatter(amount_nn, 1-final_score_FP_nn, marker='h',label='FP (Ours)',alpha=1,c='black',s=mark_size*2)


plt.scatter(amount_nn, 1-final_score_icpml_nn, marker='s',label='IC-PML',c='grey',s=mark_size,alpha=.8)
plt.scatter(amount_nn, 1-final_score_LDLE_nn,  marker='P',label='LDLE',c='purple',s=mark_size,alpha=1)

plt.scatter(amount_nn, 1-final_score_mult_tSNE_nn, marker='d', label='Multi-tSNE',c='blue',s=mark_size*1.5)


plt.scatter(amount_nn, 1-final_score_all_nn,  label='All Features',c='lime',alpha=1,s=mark_size*.8, marker= '*')
plt.scatter(amount_nn, 1-final_score_FP_raw_nn, marker='^', label='FP Features (Ours)',c='cyan',alpha=0.8,s=mark_size,  linewidths=.5, edgecolor='black')

legend= plt.legend(loc='upper right',fontsize=12,framealpha=0.8)
for handle in legend.legend_handles:
    handle.set_alpha(1)

plt.xlabel('K-Nearest Neighbors',fontsize=13)
plt.ylabel('Error',fontsize=13)
plt.title('',fontsize=15)
